# Pump & Pipe Model
In this notebook, I will explain the code that is developed for modeling the flow in pump and pipe. The pump type is NP-3085-MT3-Adaptive-462 from Xylem where the suction and discharge flanges have 80 mm diameter. The pump curve and information is uploaded on the GitHub repository, however the curves are related to the performance for transportation of water. In order to modify the model for slurry transport, some relations are introduced in the literature that are used here.

This model is supposed to simulate the material flow rate and pressure at the outlet of the pump/pipe under various loads and settings. The change in the pump's input current causes change in the speed of the pump, and the changes in the system setting results in changes in the system curve. The present model is supposed to represent the effect of these changes on the output of the system.

To run the model, first we import the required packages in Python. 

In [ ]:
import numpy as np
import math
import pandas as pd
from sklearn.linear_model import LinearRegression
import urllib.request

Due to the nonlinear behavior of the pump efficiency and input power at various flow rates, we need to find appropriate models to represent these nonlinearities using machine learning. But the implementation of machine learning requires some data. I obtained the required data (overall efficiency and input power) by taking samples from the performance curves at various speeds and flow rates. The recorded samples are saved in CSV file which is uploaded on the repository. Here we read the CSV file to obtain the data.

In [ ]:
print('Beginning file download with url...')

url = 'https://raw.githubusercontent.com/mohammadrashedi/Mohammad_codes/master/Pump%20modeling/C3085_data.csv'
urllib.request.urlretrieve(url, 'C3085_data.csv')

url = 'https://raw.githubusercontent.com/mohammadrashedi/Mohammad_codes/master/Pump%20modeling/Data_from_xylect.csv'
urllib.request.urlretrieve(url, 'Data_from_xylect.csv')

print('File download done!')
data=pd.read_csv('Data_from_xylect.csv')

In the next step, we extract the data of flow rate, head, overall efficiency, and input power of the pump to fit a model. It should be noted that the unit of the recorded data of flow rate is Liter/s and of the input power is KW. As a result, the former should be divided by 1000 and the latter should be multiplied by 1000 to convert them to the SI units.

In [ ]:
flow60=data['Flow 60'].dropna()[:,np.newaxis]/1000                 # Flow rate quantization at 60 Hz
flow55=data['Flow 55'].dropna()[:,np.newaxis]/1000                 # Flow rate quantization at 55 Hz
flow50=data['Flow 50'].dropna()[:,np.newaxis]/1000                 # Flow rate quantization at 50 Hz
flow45=data['Flow 45'].dropna()[:,np.newaxis]/1000                 # Flow rate quantization at 45 Hz
flow40=data['Flow 40'].dropna()[:,np.newaxis]/1000                 # Flow rate quantization at 40 Hz
head60=data['Head 60'].dropna()[:,np.newaxis]                      # Corresponding head samples at 60 Hz
head55=data['Head 55'].dropna()[:,np.newaxis]                      # Corresponding head samples at 55 Hz
head50=data['Head 50'].dropna()[:,np.newaxis]                      # Corresponding head samples at 50 Hz
head45=data['Head 45'].dropna()[:,np.newaxis]                      # Corresponding head samples at 45 Hz
head40=data['Head 40'].dropna()[:,np.newaxis]                      # Corresponding head samples at 40 Hz
efficiency60=data['Overall Eff 60'].dropna()[:,np.newaxis]         # Corresponding efficiency samples at 60 Hz
efficiency55=data['Overall Eff 55'].dropna()[:,np.newaxis]         # Corresponding efficiency samples at 55 Hz
efficiency50=data['Overall Eff 50'].dropna()[:,np.newaxis]         # Corresponding efficiency samples at 50 Hz
efficiency45=data['Overall Eff 45'].dropna()[:,np.newaxis]         # Corresponding efficiency samples at 45 Hz
efficiency40=data['Overall Eff 40'].dropna()[:,np.newaxis]         # Corresponding efficiency samples at 40 Hz
input_power60=data['Power input 60'].dropna()[:,np.newaxis]*1000   # Corresponding input power samples at 60 Hz
input_power55=data['Power input 55'].dropna()[:,np.newaxis]*1000   # Corresponding input power samples at 55 Hz
input_power50=data['Power input 50'].dropna()[:,np.newaxis]*1000   # Corresponding input power samples at 50 Hz
input_power45=data['Power input 45'].dropna()[:,np.newaxis]*1000   # Corresponding input power samples at 45 Hz
input_power40=data['Power input 40'].dropna()[:,np.newaxis]*1000   # Corresponding input power samples at 40 Hz

There are some constants involved in the model with the following characteristics. Here, the effective parameters for slurry flow model are also given so that we can modify the model for slurry modling easily. It should be noted that the data extracted from the pump curves represent the charactresitics from 60 Hz to 40 Hz. Based on the pump affinity laws, if the voltage remains constant, that means the model can represent the pump current variation from 2.9 A to 9.6 A. The parameter `w_s` represent the weight percentage of solid material in the fluid.

In [ ]:
d_50=0.425e-3                                                 # Diameter of solid material at 50% of size distribution
S_kt=0.8                                                      # Coefficient of solid effect
g=9.81                                                        # Gravity acceleration
h_init=2                                                      # Pump depth under the water 
Q_0=0.013                                                     # Initial flow at iteration of flow calculation
D=0.08                                                        # Pipe diameter
w_s=0                                                         # Weight percentage of solid material in the water
rho_s=2650                                                    # Density of solids
rho_w=1000                                                    # Density of fluid
rho_m=100/(w_s/rho_s+(100-w_s)/rho_w)                         # Mixture density
C_v=(rho_m-rho_w)/(rho_s-rho_w)                               # Concentration of solids by volume
L_p=30                                                        # Pipe length
mu_w=8.9e-4                                                   # Viscosity of water
mu_m=mu_w*(1+2.5*C_v+10.05*C_v**2+0.00273*np.exp(16.6*C_v))   # Mixture viscosity
eps=0.0001                                                    # Pipe roughness
V=230                                                         # Voltage
I_max=9.6                                                     # Maximum current of the pump motor
current=3                                                     # Pump current under load
I_min=(40/60)**3*I_max                                        # Minimum current
if (current<I_min) | (current>I_max):
    raise Exception('Value of current is out of range!')
I=current
freq_max=60                                                   # Frequency upper bound
freq=(I/I_max)**(1/3)*freq_max                                # Pump frequency under load  

Now, we start fitting models on the head, overall efficiency, and power input based on the flow rate. If we look at the curve representing the head vs. flow rate, it can be verified that the relation is almost linear. Hence, we use linear regression to fit the model at various frequencies.

In [ ]:
HEAD60=LinearRegression().fit(flow60,head60)
HEAD55=LinearRegression().fit(flow55,head55)
HEAD50=LinearRegression().fit(flow50,head50)
HEAD45=LinearRegression().fit(flow45,head45)
HEAD40=LinearRegression().fit(flow40,head40)

Due to the nonlinear behavior of the input power and total efficiency, it is better to fit a parsemoneous nonlinear model. By looking at the variations of overall efficiency, it is very likely that a second order (quadratic) model fits very well. On the other hand, the input power is most likely a third order model.

In [ ]:
def Parameters(Q,eff,power):    
    X1=np.ones((len(Q),1))
    X2=Q
    X3=np.multiply(Q,Q)
    X4=np.multiply(X3,Q)
    X_eff=np.column_stack((X1,X2,X3))    
    X_power=np.column_stack((X1,X2,X3,X4))
    params_1=np.matmul(np.linalg.inv(np.matmul(X_eff.T,X_eff)),np.matmul(X_eff.T,eff))
    params_2=np.matmul(np.linalg.inv(np.matmul(X_power.T,X_power)),np.matmul(X_power.T,power))
    return params_1,params_2
param_eff_60,param_pow_60=Parameters(flow60,efficiency60,input_power60)
param_eff_55,param_pow_55=Parameters(flow55,efficiency55,input_power55)
param_eff_50,param_pow_50=Parameters(flow50,efficiency50,input_power50)
param_eff_45,param_pow_45=Parameters(flow45,efficiency45,input_power45)
param_eff_40,param_pow_40=Parameters(flow40,efficiency40,input_power40)
def tot_eff(Q,param_eff):
    etta=param_eff[0]+param_eff[1]*Q+param_eff[2]*Q**2
    return etta/100
def input_power(Q,param_pow):
    inp_power=param_pow[0]+param_pow[1]*Q+param_pow[2]*Q**2+param_pow[3]*Q**3
    return inp_power

After taking these steps, we are ready to solve the differential equation governing the pump and system dynamics. Since the pump frequency changes based on the current continuously while the extracted pump data from the CSV file correspond to specific and discrete frequencies, i.e. 60, 55, 50, 45, and 40 Hz, we need to interpolate the values in between. Considering the above notes, we can solve the differential equation by iterations until convergence.

In [ ]:
Q=Q_0
dt=0.01
error=10
while error>1e-10:
    if (freq<=60) & (freq>55):
        Power=(input_power(Q,param_pow_60)-input_power(Q,param_pow_55))/5*(freq-55)+input_power(Q,param_pow_55)
        Tot_eff=(tot_eff(Q,param_eff_60)-tot_eff(Q,param_eff_55))/5*(freq-55)+tot_eff(Q,param_eff_55)
    elif (freq<=55) & (freq>50):
        Power=(input_power(Q,param_pow_55)-input_power(Q,param_pow_50))/5*(freq-50)+input_power(Q,param_pow_50)
        Tot_eff=(tot_eff(Q,param_eff_55)-tot_eff(Q,param_eff_50))/5*(freq-50)+tot_eff(Q,param_eff_50)
    elif (freq<=50) & (freq>45):
        Power=(input_power(Q,param_pow_50)-input_power(Q,param_pow_45))/5*(freq-45)+input_power(Q,param_pow_45)
        Tot_eff=(tot_eff(Q,param_eff_50)-tot_eff(Q,param_eff_45))/5*(freq-45)+tot_eff(Q,param_eff_45)
    elif (freq<=45) & (freq>=40):
        Power=(input_power(Q,param_pow_45)-input_power(Q,param_pow_40))/5*(freq-40)+input_power(Q,param_pow_40)
        Tot_eff=(tot_eff(Q,param_eff_45)-tot_eff(Q,param_eff_40))/5*(freq-40)+tot_eff(Q,param_eff_40)
    Net_Power=Power*Tot_eff
    Re=rho_m*4*Q/math.pi/D**2*D/mu_m
    f=0.25*(math.log10(eps/3.7/D+5.74/Re**0.9))**(-2)
    a_p=8*f*rho_w*L_p/math.pi**2/D**5
    b_p=S_kt*math.pi*D**2*L_p/4
    P_loss=a_p*Q**2+b_p*C_v/Q+(rho_m-rho_w)*g*h_init
    Aux=Q
    Q=Q+dt*(math.pi*D**2/4/rho_m/L_p*(Net_Power/Q-P_loss))
    error=np.abs(Aux-Q)

Finally, the fluid flow rate and the predicted head at the outlet of the pump are printed. The true head based on the linear model fitting to the pump curve is also printed and compared to the predicted head.

In [ ]:
print('The predicted flowrate is: {:.3f} l/s'.format(Q[-1]*1000))
h_predict=Net_Power/Q[-1]/rho_m/g
print('The predicted head at the slurry pump discharge is: {} m'.format(h_predict))
if (freq<=60) & (freq>55):
    HEAD=(HEAD60.predict([Q])-HEAD55.predict([Q]))/5*(freq-55)+HEAD55.predict([Q])
elif (freq<=55) & (freq>50):
    HEAD=(HEAD55.predict([Q])-HEAD50.predict([Q]))/5*(freq-50)+HEAD50.predict([Q])
elif (freq<=50) & (freq>45):
    HEAD=(HEAD50.predict([Q])-HEAD45.predict([Q]))/5*(freq-45)+HEAD45.predict([Q])
elif (freq<=45) & (freq>=40):
    HEAD=(HEAD45.predict([Q])-HEAD40.predict([Q]))/5*(freq-40)+HEAD40.predict([Q])
print('The discharge head based on pump curve is: {} m'.format(HEAD))

At the end, it worths mentioning that this code can also predict the quantities for slurry flow as long as the weigth percentage of solid material is known.